# day 7

https://adventofcode.com/2019/day/7

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day07.txt')

LOGGER = logging.getLogger('day07')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = []

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return [int(_)
                for line in fp
                for _ in line.strip().split(',')]

#### function def

In [ ]:
import aoc2019 as A, importlib
importlib.reload(A)

In [ ]:
def apply_amplifiers(intcode_program, phase_settings, inp=0, with_feedback_loop=False):
    LOGGER.debug(f'phase_settings = {phase_settings}')
    N_AMP = len(phase_settings)
    
    amplifiers = [A.IntcodeComputer(intcode=intcode_program, inputs=[p])
                  for p in phase_settings]
    amplifiers[0].inputs.append(0)

    amp_idx = 0
    while True:
        LOGGER.debug(f'processing amplifier #{amp_idx}')
        try:
            output = amplifiers[amp_idx].get_output()
        except StopIteration:
            break
        LOGGER.debug(f'amplifier #{amp_idx} output {output}')
        
        next_inp = output
        next_amp_idx = amp_idx + 1
        
        if next_amp_idx == N_AMP:
            if with_feedback_loop:
                LOGGER.debug('looping again...')
                next_amp_idx = 0
            else:
                break
        
        LOGGER.debug(f'appending output ({next_inp}) to inputs for amplifier #{next_amp_idx}')
        amplifiers[next_amp_idx].inputs.append(next_inp)    
        LOGGER.debug(f'inputs for amplifier #{next_amp_idx} are now {amplifiers[next_amp_idx].inputs}')
        
        # update shit
        amp_idx = next_amp_idx
    return output

In [ ]:
from itertools import permutations

def q_1(intcode_program, phases=None, with_feedback_loop=False):
    max_thruster_signal = 0
    max_phase_settings = None
    phases = phases or range(5)
    for phase_settings in permutations(phases, 5):
        thruster_signal = apply_amplifiers(intcode_program,
                                           phase_settings,
                                           with_feedback_loop=with_feedback_loop)
        if thruster_signal > max_thruster_signal:
            max_thruster_signal = thruster_signal
            max_phase_settings = phase_settings
    LOGGER.debug(f'max_phase_settings: {max_phase_settings}')
    LOGGER.debug(f'max_thruster_signal: {max_thruster_signal}')
    return max_thruster_signal

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    a_logger_reset_level = A.LOGGER.level
    A.LOGGER.setLevel(logging.DEBUG)
    LOGGER.warning('intcode_computer tests')
    assert list(A.compute_intcode([1002, 4, 3, 4, 33])) == []
    
    LOGGER.warning('apply_amplifiers tests')
    assert apply_amplifiers(intcode_program=[3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0],
                            phase_settings=[4, 3, 2, 1, 0]) == 43210
    assert apply_amplifiers(intcode_program=[3,23,3,24,1002,24,10,24,1002,23,-1,23,
                                             101,5,23,23,1,24,23,23,4,23,99,0,0],
                            phase_settings=[0,1,2,3,4]) == 54321
    assert apply_amplifiers(intcode_program=[3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,
                                             1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0],
                            phase_settings=[1,0,4,3,2]) == 65210
    #assert q_1(test_data) == True
    A.LOGGER.setLevel(a_logger_reset_level)
    LOGGER.setLevel(logging.WARN)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(intcode_program):
    return q_1(intcode_program,
               phases=range(5, 10),
               with_feedback_loop=True)

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    LOGGER.warning('intcode_computer tests')
    assert list(A.compute_intcode([1002, 4, 3, 4, 33])) == []
    
    LOGGER.warning('apply_amplifiers tests')
    assert apply_amplifiers(intcode_program=[3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
                                             27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5],
                            phase_settings=[9, 8, 7, 6, 5],
                            with_feedback_loop=True) == 139_629_729
    assert apply_amplifiers(intcode_program=[3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
                                             -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
                                             53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10],
                            phase_settings=[9, 7, 8, 5, 6],
                            with_feedback_loop=True) == 18_216
    
    LOGGER.setLevel(logging.WARN)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin